## Fine Tuning a Llama2 as a chat model

<p>After pretraining learning make more series from Andrej Karpathy. </p>
<p>Learning how we can do Fine Tuning using this video from Krish Naik https://www.youtube.com/watch?v=iOdFUJiB0Zc</p>

In [ ]:
# !pip install -U bitsandbytes

In [ ]:
# !pip uninstall bitsandbytes -y
# !pip install bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 83.1 MB/s  0:00:00m eta 0:00:01


In [ ]:
# !pip install --upgrade pip setuptools wheel


  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.2 MB/s  0:00:00
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 78.1.1
    Uninstalling setuptools-78.1.1:
      Successfully uninstalled setuptools-78.1.1 0/2 [setuptools]
  Attempting uninstall: pip━━━━━━━━━━━━━━━━━━━━━ 0/2 [setuptools]
    Found existing installation: pip 25.2━━━ 0/2 [setuptools]
    Uninstalling pip-25.2:━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [pip]
      Successfully uninstalled pip-25.2━━━━━━━━━━━━━━━━━━━ 1/2 [pip]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pip]1/2 [pip]


In [ ]:
# !pip install transformers==4.41.2 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 48.4 MB/s  0:00:00 eta 0:00:01
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
  Attempting uninstall: transformers━━━━━━━━━━━━ 0/2 [huggingface_hub]
    Found existing installation: transformers 4.41.12m0/2 [huggingface_hub]
    Uninstalling transformers-4.41.1:╺━━━━━━━━━━━━━━━━━━━ 1/2 [transformers]
      Successfully uninstalled transformers-4.41.1━━━━━━━━━━━━━━━━ 1/2 [transformers]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.4.1 requires huggingface-hub<2.0,>=0.25.0, but you have huggingface-hub 0.23.4 which is incompatible.
peft 0.17.1 requires huggingface_hub>=0.25.0, b

In [ ]:
!pip install -U \
  transformers==4.41.2 \
  huggingface-hub==0.26.5 \
  tokenizers==0.19.1


  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.2.3
    Uninstalling huggingface_hub-1.2.3:
      Successfully uninstalled huggingface_hub-1.2.3


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade tokenizers==0.19.1

In [ ]:
# import tokenizers
# tokenizers.__version__

'0.22.1'

In [ ]:
# import sys
# !{sys.executable} -m pip install --upgrade tokenizers==0.19.1

In [26]:
!pip install -U \
  transformers \
  peft \
  trl \
  huggingface_hub \
  tokenizers


  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-1.2.3-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
Using cached huggingface_hub-0.36.0-py3-none-any.whl (566 kB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 11.4 MB/s  0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.26.5
    Uninstalling huggingface-hub-0.26.5:
      Successfully uninstalled huggingface-hub-0.26.5
  Attempting uninstall: tokenizers━━━━━━━━━━━━━━ 0/5 [huggingface_hub]
    Found existing installation: tokenizers 0.19.10/5 [huggingface_hub]
    Uninstalling tokenizers-0.19.1:━━━━━━━━━ 0/5 

In [27]:
import tokenizers
tokenizers.__version__

'0.19.1'

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, # To load the Causal lang. Model like ChatGPT
    AutoTokenizer, # To load the tokenization
    BitsAndBytesConfig, # for low memory convert to low precision like FP32 --> FP16
    HfArgumentParser,
    TrainingArguments, # passing Training arguments
    pipeline,
    logging,
)
from transformers import LlamaTokenizer
from peft import LoraConfig, PeftModel # Parameter efficient Fine Tuning. LORA performs matrix decomposition which makes the matrix quite small. There is QLORA which works on Quantization LORA.
from trl import SFTTrainer # Train transformer language models with reinforcement learning.

/mnt/data/tools/miniconda3/envs/torch_gpu_dna/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/data/tools/miniconda3/envs/torch_gpu_dna/lib/python3.10/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


- Orignal Dataset: https://huggingface.co/datasets/timdettmers/openassistant-guanaco
- Reformat Dataset following the Llama 2 template with 1k sample: https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k
- Complete Reformat Dataset following the Llama 2 template: https://huggingface.co/datasets/mlabonne/guanaco-llama2

#**How to fine tune Llama 2**
- We also need to consider the overhead due to optimizer states, gradients, and forward activations
- Full fine-tuning is not possible here: we need parameter-efficient fine-tuning (PEFT) techniques like LoRA or QLoRA.
- To drastically reduce the VRAM usage, we must fine-tune the model in 4-bit precision, which is why we’ll use QLoRA here.

1. Load a llama-2-7b-chat-hf model (chat model)
2. Train it on the mlabonne/guanaco-llama2-1k (1,000 samples), which will produce our fine-tuned model Llama-2-7b-chat-finetune

QLoRA will use a rank of 64 with a scaling parameter of 16. We’ll load the Llama 2 model directly in 4-bit precision using the NF4 type and train it for one epoch


In [5]:
# The model that you want to train from the Hugging Face hub
# Non-chat version → worse conversational behavior unless heavily fine-tuned.
model_name = "NousResearch/Llama-2-7b-chat-hf"


# The instruction dataset to use
# Small (~1k examples) → good for quick adaptation, not general knowledge learning.
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64 ## This is the rank by which matrix decomposition takes place. 
# suppose if lora_r = 1 then 3 X 3 matrix is decomposed to 3 X 1 and 1 X 3.

# Alpha parameter for LoRA scaling
# Scaling factor.
# Effective LoRA contribution ≈ alpha / r = 16 / 64 = 0.25
lora_alpha = 16 # scaling factor

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Even though your model weights are stored in 4-bit, the actual matrix multiplications are NOT done in 4-bit.
# Instead:
# Weights are dequantized on the fly 
# computation happens in float16
# 4-bit weights  →  dequantize →  float16 compute → output


# Quantization type (fp4 or nf4)
# NF4 (NormalFloat4) is optimized for neural network weights.
# Better accuracy than FP4. 
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Original fp16 model
#         ↓
# Quantized to NF4 (4-bit)   ← bnb_4bit_quant_type="nf4"
#         ↓
# Stored efficiently in GPU memory
#         ↓
# Dequantized to fp16        ← bnb_4bit_compute_dtype="float16"
#         ↓
# Matrix multiplication & gradients


################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True # (half precision) 1 bit sign, 5 bits exponent, 10 bits mantissa (fraction)
bf16 = False # 1 bit sign, 8 bits exponent, 7 bits mantissa
# Same exponent range as fp32
# Much more numerically stable than fp16
# Lower precision than fp16
# Large-scale model training (TPUs, modern GPUs like A100, H100)

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1
# (forward → backward) × N → optimizer.step() where N is the number of minibatches 

# Enable gradient checkpointing
gradient_checkpointing = True
# Training transformers normally stores all intermediate activations so gradients can be computed during backprop.
# For a 7B model, this eats huge GPU memory.
# Gradient checkpointing:
# Does NOT store all activations
# Recomputes them during backward pass instead
# Trades compute time for memory savings
# so instead of Forward → store activations → Backward
# It do
# Forward → discard activations
# Backward → recompute forward → compute gradients


# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001
# Weight decay is a regularization technique that prevents a model from overfitting by discouraging weights from becoming too large.
# Think of it as telling the model:
# “Fit the data well, but don’t rely on extremely large parameter values.”
# During training, models can reduce loss by:
# Making some weights very large
# Memorizing training examples
# Weight decay penalizes large weights, pushing them slightly toward zero at every update.
# Without weight decay:
# w = w - lr * gradient
# With weight decay:
# w = w - lr * (gradient + λ * w)

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"
# Early training → need large steps to learn quickly
# Later training → need small steps to fine-tune
# Too large LR → divergence
# Too small LR → slow or stuck training

# Number of training steps (overrides num_train_epochs)
max_steps = -1


# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

#### Since Llama 2 tokenizer does not work on the newer version. 
So we have saved the tokenizer on the older version and updated all the packages. 

In [ ]:
# Load LLaMA tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, legacy=True)

In [ ]:
# tokenizer

LlamaTokenizerFast(name_or_path='NousResearch/Llama-2-7b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	32000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}

In [ ]:
# os.chdir("/mnt/data/projects/.immune/Personal/Transformers_Tutorial/Llama/")

In [ ]:
# save_dir = "./llama2_tokenizer_frozen"
# tokenizer.save_pretrained(save_dir)

('./llama2_tokenizer_frozen/tokenizer_config.json',
 './llama2_tokenizer_frozen/special_tokens_map.json',
 './llama2_tokenizer_frozen/tokenizer.model',
 './llama2_tokenizer_frozen/added_tokens.json',
 './llama2_tokenizer_frozen/tokenizer.json')

1. First of all, we want to load the dataset we defined. Here, our dataset is already preprocessed but, usually, this is where you would reformat the prompt, filter out bad text, combine multiple datasets, etc.

2. Then, we’re configuring bitsandbytes for 4-bit quantization.

3. Next, we're loading the Llama 2 model in 4-bit precision on a GPU with the corresponding tokenizer.

4. Finally, we're loading configurations for QLoRA, regular training parameters, and passing everything to the SFTTrainer. The training can finally start!

In [ ]:
# Load dataset 
dataset = load_dataset(dataset_name, split="train") ## Dataset to load

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype) # bnb_4bit_compute_dtype = float 16

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit, # Quantization is performed at 4 bits
    bnb_4bit_quant_type=bnb_4bit_quant_type, # nf4 that is more precised
    bnb_4bit_compute_dtype=compute_dtype, # float 16 caluclation is perform with fp16 for better accuracy
    bnb_4bit_use_double_quant=use_nested_quant, # use_nested_quant = False
)

In [ ]:
# !python -c "import bitsandbytes as bnb; print(bnb.__version__)"

0.49.0


In [7]:
use_4bit

True

In [ ]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    print(major)
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)
    else:
        print("Your GPU does not supports bfloat16: accelerate training with bf16=True")
# CUDA compute capability:
# 7.x → Volta / Turing (V100, T4)
# 8.0+ → Ampere (A100)
# 9.0+ → Hopper (H100)


7
Your GPU does not supports bfloat16: accelerate training with bf16=True


In [10]:
print(model_name, bnb_config, device_map)

NousResearch/Llama-2-7b-chat-hf BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_storage": "uint8",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": false,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}
 {'': 0}


In [11]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,  # Load the entire model on the GPU 0
    trust_remote_code=True
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.57s/it]


In [12]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (

In [13]:
model.config

LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "dtype": "float16",
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "float16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  

In [14]:
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
# !pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.4 MB/s  0:00:00


In [ ]:
# since we saved using old version, we just need to upload it on the newer version.
from transformers import AutoTokenizer
os.chdir("/mnt/data/projects/.immune/Personal/Transformers_Tutorial/Llama/")

tokenizer = AutoTokenizer.from_pretrained(
    "llama2_tokenizer_frozen",
    use_fast=True
)

In [23]:
tokenizer

LlamaTokenizerFast(name_or_path='llama2_tokenizer_frozen', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	32000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}
)

In [24]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training


In [25]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
# Set training parameters
# https://huggingface.co/docs/transformers/v5.0.0rc1/en/main_classes/trainer#transformers.TrainingArguments

print(
    output_dir, num_train_epochs, per_device_train_batch_size, gradient_accumulation_steps,
    optim,save_steps,logging_steps,learning_rate,weight_decay,fp16,bf16,max_grad_norm,max_steps,
    warmup_ratio,group_by_length,lr_scheduler_type)


./results 1 4 1 paged_adamw_32bit 0 25 0.0002 0.001 True False 0.3 -1 0.03 True cosine


In [29]:
# Set training parameters
# https://huggingface.co/docs/transformers/v5.0.0rc1/en/main_classes/trainer#transformers.TrainingArguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)


In [34]:
training_arguments

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=True,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=no,
eval_use_gather_object=False,
fp16=True,
fp16_ba

In [31]:
print(dataset)
print(peft_config)
print(packing)

Dataset({
    features: ['text'],
    num_rows: 1000
})
LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, peft_version='0.18.0', base_model_name_or_path=None, revision=None, inference_mode=False, r=64, target_modules=None, exclude_modules=None, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, alora_invocation_tokens=None, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None, arrow_config=None, ensure_weight_tying=False)
False


In [41]:
dataset['text'][0:10]

['<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país. </s>',
 '<s>[INST] Самый великий человек из всех живших на планете? [/INST] Для начала нужно выбрать критерии величия человека. Обычно великим

In [48]:
training_arguments

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=True,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=no,
eval_use_gather_object=False,
fp16=True,
fp16_ba

In [46]:
import trl
trl.__version__

'0.26.2'

In [52]:
# Set supervised fine-tuning (SFT) parameters
# https://huggingface.co/docs/trl/en/sft_trainer#expected-dataset-type-and-format
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_arguments,
)

/mnt/data/tools/miniconda3/envs/torch_gpu_dna/lib/python3.10/site-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/mnt/data/tools/miniconda3/envs/torch_gpu_dna/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
# !pip install tensorboardX

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [tensorboardX] [tensorboardX]


In [54]:
dir(trainer)

['__backends',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activate_neftune',
 '_add_sm_patterns_to_gitignore',
 '_align_special_tokens',
 '_created_lr_scheduler',
 '_deactivate_neftune',
 '_determine_best_metric',
 '_evaluate',
 '_finish_current_push',
 '_fsdp_qlora_plugin_updates',
 '_gather_and_numpify',
 '_get_collator_with_removed_columns',
 '_get_dataloader',
 '_get_eval_sampler',
 '_get_learning_rate',
 '_get_num_items_in_batch',
 '_get_output_dir',
 '_get_train_sampler',
 '_hp_search_setup',
 '_inner_training_loop',
 '_is_attention_mask_causal',
 '_is_vision_dataset',
 '_is_vlm',
 '_issue_warnings_after_load',
 '_load_best_model',
 '_loa

In [55]:
# Train model
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 0 has a total capacity of 14.57 GiB of which 58.75 MiB is free. Including non-PyTorch memory, this process has 14.51 GiB memory in use. Of the allocated memory 14.02 GiB is allocated by PyTorch, and 373.51 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [56]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Thu Dec 25 08:28:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P0             26W /   70W |   14859MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----